Install required libraries

In [3]:
pip install ijson --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 2.1 MB/s eta 0:00:00


In [4]:
pip install -U sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00


In [5]:
#mount the google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#import required libraries

import ijson
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import numpy as np
import pandas as pd

In [7]:
#base path where all project files are stored

base_path = "drive/MyDrive/CS646"

In [8]:
#analysing training data

training_data_users = {}
training_data_user_ids = []
count = 0

with open(base_path + '/train_questions.json', 'r') as file:
  parser = ijson.items(file, 'item')
  for item in parser:
    user_id = item.get('id')
    user_profile = item.get('profile')
    num_profiles = len(user_profile)

    training_data_users[user_id] = num_profiles
    training_data_user_ids.append(user_id)

    count += 1

In [9]:
print("training data statistics")
print("length of training data: ", count)
print("max number of profiles for a user: ", max(val for key,val in training_data_users.items()))
print("min number of profiles for a user: ", min(val for key,val in training_data_users.items()))

training data statistics
length of training data:  20000
max number of profiles for a user:  1028
min number of profiles for a user:  99


In [10]:
#analysing test data

test_data_users = {}
test_data_user_ids = []
count = 0

with open(base_path + '/test_questions.json', 'r') as file:
  parser = ijson.items(file, 'item')
  for item in parser:
    user_id = item.get('id')
    user_profile = item.get('profile')
    num_profiles = len(user_profile)

    test_data_users[user_id] = num_profiles
    test_data_user_ids.append(user_id)

    count += 1

In [11]:
print("test data statistics")
print("length of test data: ", count)
print("max number of profiles for a user: ", max(val for key,val in test_data_users.items()))
print("min number of profiles for a user: ", min(val for key,val in test_data_users.items()))

test data statistics
length of test data:  2500
max number of profiles for a user:  1006
min number of profiles for a user:  99


In [12]:
#analysing validation data

val_data_users = {}
val_data_user_ids = []
count = 0

with open(base_path + '/dev_questions.json', 'r') as file:
  parser = ijson.items(file, 'item')
  for item in parser:
    user_id = item.get('id')
    user_profile = item.get('profile')
    num_profiles = len(user_profile)

    val_data_users[user_id] = num_profiles
    val_data_user_ids.append(user_id)

    count += 1

In [13]:
print("val data statistics")
print("length of val data: ", count)
print("max number of profiles for a user: ", max(val for key,val in val_data_users.items()))
print("min number of profiles for a user: ", min(val for key,val in val_data_users.items()))

val data statistics
length of val data:  2500
max number of profiles for a user:  1023
min number of profiles for a user:  99


Assert that there is nothing common between training, test and validation data

In [14]:
list(set(training_data_user_ids) & set(test_data_user_ids))

[]

In [15]:
list(set(val_data_user_ids) & set(test_data_user_ids))

[]

In [16]:
list(set(val_data_user_ids) & set(training_data_user_ids))

[]

In [17]:
# This functions creates clusters for a given user profile. It puts reviews with same rating in a single cluster

def get_clusters_for_user(user_profiles):
    clusters = {}
    clusters['1'] = []
    clusters['2'] = []
    clusters['3'] = []
    clusters['4'] = []
    clusters['5'] = []

    for profile in user_profiles:
        text = profile.get('text')
        score = profile.get('score')
        clusters[score].append(text)

    return clusters

In [18]:
#initiate the model used for getting embeddings

%%capture
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
#method to get the scores for each cluster

def get_scores_for_clusters(test_review, clusters):
    scores = []
    top_reviews = []
    for score in ('1', '2', '3', '4', '5'):
        reviews = clusters.get(score)
        # print("total score " + score + " reviews ", len(reviews))

        cluster_score = 0
        count = 0
        highest_score = 0
        top_review = ""

        test_review_embedding = model.encode([test_review])

        for review in reviews:
            review_embedding = model.encode([review])
            score = util.pytorch_cos_sim(review_embedding, test_review_embedding).item()
            cluster_score += score
            count += 1

            if highest_score < score:
                highest_score = score
                top_review = review
            if count>=200:
                break

        if count > 0:
            cluster_avg_score = cluster_score/count
        else:
            cluster_avg_score = 0

        scores.append(cluster_avg_score)
        top_reviews.append(top_review)

    return scores, top_reviews


In [ ]:
# get review with highest score in a given cluster

def get_top_review_in_cluster(test_review, reviews):
  test_review_embedding = model.encode([test_review])
  max_score = 0
  top_score_review = ""

  for review in reviews:
      review_embedding = model.encode([review])
      score = util.pytorch_cos_sim(review_embedding, test_review_embedding).item()
      if max_score < score:
        max_score = score
        top_score_review = review

  return top_score_review

In [ ]:
# get k reviews by fetching 1 top review from each of k clusters

def get_top_k_reviews(clusters_scores, top_reviews, k):
  top_k_reviews = {}
  sorted_indices = np.argsort(clusters_scores)[::-1]

  for i in range(k):
    if clusters_scores[sorted_indices[i]] != 0:
      score = str(sorted_indices[i]+1)
      review = top_reviews[sorted_indices[i]]
      top_k_reviews[score] = review

  return top_k_reviews

In [ ]:
# method to generate the prompt for LLM

def generate_llm_input(input, top_k_reviews):
  prompt = ""
  for score,review in top_k_reviews.items():
    prompt += score + " is the score for " + '"' + review + '"' + ", and "

  prompt = prompt[:-6]
  prompt += input

  return prompt

In [ ]:
# get language model score for the given input review and existing relevant reviews

def get_language_model_score(input, top_k_reviews):
  model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
  tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

  prompt = generate_llm_input(input, top_k_reviews)
  # print("prompt is :" + prompt)

  inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
  outputs = model.generate(**inputs, max_new_tokens=20)
  rating = tokenizer.batch_decode(outputs, skip_special_tokens=True)

  return rating

In [ ]:
# function to run the model on test data and generate the outputs

test_ratings = {}

def test():
    count = 0
    with open(base_path + '/test_questions.json', 'r') as file:
        parser = ijson.items(file, 'item')
        for item in parser:
            user_id = item.get('id')
            user_profiles = item.get('profile')
            input = item.get('input')

            clusters = get_clusters_for_user(user_profiles)

            test_review = input.split(':')[1].strip()

            clusters_scores, top_reviews = get_scores_for_clusters(test_review, clusters)

            top_k_reviews = get_top_k_reviews(clusters_scores, top_reviews, 3)

            rating = get_language_model_score(input, top_k_reviews)

            test_ratings[user_id] = rating[0]
            print("rating is " + rating[0])


In [ ]:
# calling test function to get the outputs of test dataset. Storing in a csv file

# test()
# df = pd.DataFrame(test_ratings.items(), columns=['id', 'rating'])
# df.to_csv(base_path + '/test_ratings.csv')

In [ ]:
# function to run the model on validation data and generate the outputs.
# this method is different from the test function above as it predicts the outputs in batches

def val(start_index, end_index):
    val_ratings = {}
    count = 0
    iterated = 0
    invalid_ratings = 0
    with open(base_path + '/dev_questions.json', 'r') as file:
        parser = ijson.items(file, 'item')
        for item in parser:
            count += 1
            if count<start_index or count>end_index:
                continue
            user_id = item.get('id')
            user_profiles = item.get('profile')
            input = item.get('input')

            clusters = get_clusters_for_user(user_profiles)

            test_review = input.split(':')[1].strip()

            clusters_scores, top_reviews = get_scores_for_clusters(test_review, clusters)

            top_k_reviews = get_top_k_reviews(clusters_scores, top_reviews, 2)

            rating = get_language_model_score(input, top_k_reviews)
            rating = rating[0]
            if rating not in ('1', '2', '3', '4', '5'):
                rating = 5
                invalid_ratings += 1

            val_ratings[user_id] = rating

            iterated += 1
            if iterated%10 == 0:
                print("iterated: " + str(iterated))

    return val_ratings


In [ ]:
#running in multiple batches one by one due to resource constraint

#batch1(1,400)
#val_ratings = val(1,400)

#batch1(401,800)
#val_ratings = val(401,800)

#batch1(801,1200)
#val_ratings = val(801,1200)

#batch1(1201,1600)
#val_ratings = val(1201,1600)

#batch1(1601,2000)
#val_ratings = val(1601,2000)

#batch1(2001,2500)
#val_ratings = val(2001,2500)

In [ ]:
#for each of the batches, storing the predictions obtained in a csv file

df = pd.DataFrame(val_ratings.items(), columns=['id', 'rating'])
df.tail(10)
df.to_csv(base_path + '/val_ratings_batch_5_1601_2000.csv')

Metric evaluation part

In [ ]:
# loading the predictions stored in multiple csv files. The predicted ratings are stored in y_predicted

y_predicted = []

filenames = ['val_ratings_batch_1_1_400.csv', 'val_ratings_batch_2_401_800.csv', 'val_ratings_batch_3_801_1200.csv', 'val_ratings_batch_4_1201_1600.csv', 'val_ratings_batch_5_1601_2000.csv',
             'val_ratings_batch_6_2001_2500.csv']

for filename in filenames:
    filepath = base_path + '/' + filename
    with open(filepath, 'r') as f:
        count = 0
        for line in f:
            if count != 0:
                y_predicted.append(int(line.split(',')[2]))
            count += 1


In [ ]:
# assert that length of predicted values is 2500 (length of validation data)

len(y_predicted)

2500

In [ ]:
# load the ground truth values from the json file. The ground truth ratings are stored in y_actual

import json

y_actual = []
with open(base_path + '/dev_outputs.json') as f:
    d = json.load(f)
    golds = d.get('golds')
    for gold in golds:
        y_actual.append(int(gold.get('output')))
    print("y_actual ", y_actual)


y_actual  [2, 5, 5, 1, 5, 3, 4, 4, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 3, 5, 4, 5, 5, 4, 5, 4, 2, 3, 5, 2, 4, 5, 4, 3, 5, 5, 5, 5, 5, 4, 4, 1, 5, 5, 4, 5, 4, 5, 4, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 4, 5, 4, 1, 4, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 3, 5, 5, 1, 5, 3, 3, 4, 5, 4, 4, 3, 4, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 5, 4, 2, 5, 5, 4, 5, 4, 5, 5, 5, 4, 5, 4, 5, 3, 4, 3, 3, 5, 5, 4, 4, 5, 4, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 4, 3, 4, 4, 5, 5, 3, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 1, 5, 5, 5, 1, 5, 5, 5, 4, 4, 3, 5, 5, 4, 4, 1, 4, 3, 5, 3, 2, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3, 5, 4, 2, 4, 4, 4, 1, 5, 5, 5, 4, 5, 4, 5, 5, 2, 5, 3, 4, 5, 5, 4, 5, 3, 5, 5, 1, 2, 5, 4, 3, 5, 2, 5, 2, 5, 5, 5, 4, 4, 4, 5, 4, 5, 5, 5, 5, 5, 2, 5, 5, 4, 5, 4, 5, 5, 4, 5, 5, 5, 4, 5, 4, 4, 5, 5, 1, 5, 4, 4, 5, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 4, 5, 4, 5, 5, 5, 3, 4, 4, 3, 4, 5, 4, 5, 5, 5, 1, 5, 1, 4, 3, 5, 4, 2, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 3, 5, 5, 3, 5, 5, 2, 5,

In [ ]:
# calculate the Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE)

from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

rms = sqrt(mean_squared_error(y_actual, y_predicted, squared=False))
mae = mean_absolute_error(y_actual, y_predicted)

print("rms ", rms)
print("mae ", mae)

rms  1.0531161619882878
mae  0.7004
